<a href="https://colab.research.google.com/github/HackerCupAI/starter-kits/blob/main/rag/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{rag-hackercup} -->

<img src="http://wandb.me/logo-im-png" width="400" alt="Weights & Biases" />


# Introduction


</a>


In this notebook, we will build a few Code Generation agents for the [HackerCup AI](https://hackercupai.github.io/) challenge.

We will build three different agents using different techniques and evaluate them using [W&B Weave](https://weave-docs.wandb.ai/).


<img src="https://raw.githubusercontent.com/wandb/weave/master/docs/static/img/evals-hero.png" width="800" height="450">

A more detailed walkthough of the approach we will use in this notebook can be found in the following Youtube video:
Hint: Click on the image to watch the video 😎

<a target="_blank" href="https://www.youtube.com/watch?v=cObBj2UpWK8">
<img src="https://img.youtube.com/vi/cObBj2UpWK8/0.jpg" width="600" height="450">
</a>

## Weave


Weave is a lightweight toolkit for tracking and evaluating LLM applications, built by Weights & Biases. We will use the following weave to trace and evaluate the various agents we build.

We will use Weave to keep track and evaluate the different agents we build.

Our goal is to bring rigor, best-practices, and composability to the inherently experimental process of developing AI applications, without introducing cognitive overhead.

If you want to learn more about Weave, you can [get started](https://weave-docs.wandb.ai/quickstart) by decorating Python functions with `@weave.op`.

## Setup 

**Note: You need to run this cell only once**
We will clone the starter-kits repo
Set the rag folder as our working directory
and install the dependencies for the project.

**You can comment out the cell after you have run it once.**

In [1]:
# # Clone the starter-kits repo
# !git clone https://github.com/tcapelle/hackercup_rag
# # Change directory to the rag folder. Running the next line twice in the same session will raise an error.
# %cd hackercup_rag
# # Install dependencies
# !pip install -r requirements.txt

In [4]:
import weave

WEAVE_PROJECT = "hackercup"
weave_client = weave.init(WEAVE_PROJECT)

Logged in as Weights & Biases user: capecape.
View Weave data at https://wandb.ai/capecape/hackercup/weave


## Dataset
We will use [HackerCup dataset](https://huggingface.co/datasets/hackercupai/hackercup) in this notebook.

Specifically, the **practice** dataset from the **2023** season.

We have already processed the dataset and saved it as a [`weave.Dataset`](https://weave-docs.wandb.ai/guides/core-types/datasets/). You can either use the Dataset by running the next cell or download the dataset using the instructions below.

We will use the dataset to load some practice problems and solutions from the HackerCup dataset and evaluate our agents on it.

In [5]:
from typing import Any

from utils import (
    Problem,
    Solution,
    check_correctness,
    async_client,
    FAST_LLM,
    STRONG_LLM,
    format_response,
)


practice_dataset_uri = "weave:///parambharat/hackercup/object/practice_dataset:R35fXf9N3FE2IOesg7bRPaPAxiE9YbpirhXO9HcHs8w"
problems_dataset = weave.ref(practice_dataset_uri).get().rows[:]
problems = list(map(lambda x: Problem(**x), problems_dataset))
problem = problems[0]
print("Sample Problem:\n\n", problem.model_dump_json(indent=2))

Sample Problem:

 {
  "problem_dir": "data/2023/practice",
  "problem_name": "two_apples_a_day",
  "problem_description": "“An apple a day keeps the doctor away” is Steve’s motto. His other motto, “You can never have too much of a good thing,” holds true for both apples and mottos. Steve would like to eat two apples per day for the next \\(N\\) days, but with strict adherence to his third motto “Consistency is key.” Specifically, he’d like the sum of the two apple weights he eats over the next \\(N\\) days to be the same for each day.\n\nSteve has already purchased \\(2*N-1\\) apples, the \\(i\\)th of which weighs \\(A_i\\) ounces. He'd like to buy one more apple that's as light as possible to fulfill his goal. Steve can buy an apple of any positive integer weight in ounces from the store. Is it possible for him to reach his goal, and if so, what weight apple should he buy?\n\n{{PHOTO_ID:1563872647765708|WIDTH:600}}\n\n\n*The above image depicts the solution to the first sample. Each d

Alternatively, you can download the dataset by running the download script from the [submit-first-solution](https://github.com/HackerCupAI/starter-kits/tree/main/submit_first_solution). Specifically, you can run the following command to download the dataset:

```bash
python download.py --year 2023 --dataset_folder data
```


This should create a `dataset` folder with the problems and solutions.

Here's an example of what the data looks like for the `dim_sum_delivery` problem from the `2023` season:

```
data/dataset/2023/practice
...
├── dim_sum_delivery.cpp
├── dim_sum_delivery.in
├── dim_sum_delivery.md
├── dim_sum_delivery.out
├── dim_sum_delivery_sample_input.txt
├── dim_sum_delivery_sample_output.txt
├── dim_sum_delivery_sol.md
...
```

Each problem has a `in`, `out`, `md`, `cpp`, and `sol` file.

The `in` file contains the input data for the problem.
The `out` file contains the expected output for the problem.
The `md` file contains the problem statement.
The `cpp` file contains the source code to the solution.
The `sol` file contains the detailed solution to the problem.
The `sample_input.txt` and `sample_output.txt` files contain the sample input and output for the problem. These are the test cases that will be available to the agent during development and evaluation.

In [6]:
import asyncio
import logging

from nest_asyncio import apply

apply()

# Some logging to see the progress
logging.basicConfig(
    format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO
)

logger = logging.getLogger(__name__)

## Zero-shot Agent

For our first agent, we will use a `zero-shot solver`.
It's a simple LLM API call with a detailed prompt to solve the problem.

But first we need to load the problems and convert them to a more structured format and define a way to run the code and evaluate the solution.

First we'll start with loading some utilities. While there are other utilities we load, the ones we care about the most are `load_problem` and `check_correctness`.

The `load_problem` function will load a problem from our dataset into a more structured format.
The `check_correctness` function will run the generated code and evaluate the solution against the expected output for the sample test cases.

In [7]:
# Simple check to see if the code evaluation works
# We will use this to check the programs our the agents generate

program_code = "print('hello, world!')"
input_data = ""
expected_output = "hello, world!"
timeout = 2

test_result = check_correctness(program_code, input_data, expected_output, timeout)
print("Example 1: ", test_result)
test_result = check_correctness("print('goodbye')", input_data, "hi there", timeout)
print("Example 2: ", test_result)

🍩 https://wandb.ai/capecape/hackercup/r/call/0191c2b4-7353-7d22-910d-3829380a63af
Example 1:  passed
🍩 https://wandb.ai/capecape/hackercup/r/call/0191c2b4-772c-71f2-a36d-afaaaf72f876
Example 2:  WRONG ANSWER!!

<expected>
'hi there'
</expected>
---
<got>
'goodbye
'
</got>


Now that we have a way to load a problem and evaluate a solution, let's define a prompt to solve the problem and create a simple agent to solve the problem. 

Here'e one such prompt we will use to solve the problem, it contains instructions for the model on how to solve the problem and the format of the response we expect from the model. Feel free to tweak the prompt if you like but this should work decently well for our use case.

In [8]:
from agent import SOLVER_INSTRUCTIONS

print(SOLVER_INSTRUCTIONS)

2024-09-05 17:01:33,464 : INFO : PyTorch version 2.4.0 available.


You are a world-class competitive programmer tasked with solving a programming problem. 
You will be provided with a problem statement, and you need to create a Python3 solution for it. 
Your task it to develop a winning solution to the problem in Python3 programming language.
You will do this in a step-by-step manner.

Step 1: Extract the core question and the problem-solving information from the problem statement.
Step 2: Describe the algorithm used to solve the problem.
Step 3: Write a short tutorial on the algorithm and how it works.
Step 4: Generate a step by step plan to solve the problem.
Step 5: Generate the pseudocode to solve the problem.
Step 6: Write the final solution in Python3 programming language to solve the problem.

Competition Guidelines:
    a. Do not use any external libraries; stick to Python 3 standard library
    b. Handle input and output using standard input/output (stdin/stdout)
    c. Use helper functions to improve readability of the code.
    c. Use the `

**Note**: Here we have defined a `Solution` model to enforce the format of the response we expect from the model.
If you change the `SOLVER_INSTRUCTIONS`, you need to change the `Solution` model to enforce the new format.
We use `format_response` to enforce the format of the response we expect from the model.

In [9]:
@weave.op
async def draft_solution(
        problem: Problem, model: str = FAST_LLM, temperature: float = 0.0
) -> Solution:
    user_prompt = f"""{problem.as_xml}
---
Let's think step by step to solve the problem:
"""

    response = await async_client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": SOLVER_INSTRUCTIONS},
            {"role": "user", "content": user_prompt},
        ],
        response_model=None,
        temperature=temperature,
    )
    formatted_response = await format_response(
        response.choices[0].message.content, Solution
    )
    return formatted_response

With the main solution drafter ready, we can define the `zero_shot_solver` agent.
The agent will use the `draft_solution` function to draft a solution and the `check_correctness` function to check the correctness of the generated solution and return the result.



In [10]:
@weave.op
async def zero_shot_solver(
        problem: Problem, model: str = FAST_LLM, temperature: float = 0.0, timeout: int = 10
) -> dict:
    logger.info("Drafting intial zero-shot solution")
    solution = await draft_solution(
        problem=problem,
        model=model,
        temperature=temperature,
    )
    test_report = check_correctness(
        solution.source_code, problem.sample_input, problem.sample_output, timeout
    )
    logger.info(f"Draft solution result: {repr(test_report)}")
    return {"solution": solution, "test_report": test_report, "stage": "zero-shot"}

In [11]:
# test the zero-shot agent on the sample problem
zero_shot_result = await zero_shot_solver(problem)
print("*" * 80)
print(zero_shot_result["solution"].source_code)
print("*" * 80)
print(zero_shot_result["test_report"])

2024-09-05 17:01:35,636 : INFO : Drafting intial zero-shot solution
17:01:38 - LiteLLM:INFO: utils.py:2975 - 
LiteLLM completion() model= open-mistral-nemo-2407; provider = mistral
2024-09-05 17:01:38,577 : INFO : 
LiteLLM completion() model= open-mistral-nemo-2407; provider = mistral
2024-09-05 17:01:45,482 : INFO : HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
17:01:46 - LiteLLM:INFO: utils.py:2975 - 
LiteLLM completion() model= open-mistral-nemo-2407; provider = mistral
2024-09-05 17:01:46,136 : INFO : 
LiteLLM completion() model= open-mistral-nemo-2407; provider = mistral
2024-09-05 17:01:51,482 : INFO : HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-05 17:01:51,974 : INFO : Draft solution result: "WRONG ANSWER!!\n\n<expected>\n'Case #1: 4\nCase #2: 7\nCase #3: 1\nCase #4: -1\nCase #5: 6\nCase #6: -1\nCase #7: 1000000002\n'\n</expected>\n---\n<got>\n'Case #1: 1\nCase #2: 3\nCase #3: 1\nCase #4: 2\nCase #5: 6

********************************************************************************
T = int(input())
for t in range(1, T + 1):
    N = int(input())
    A = list(map(int, input().split()))
    total_weight = sum(A)
    remainder = total_weight % (2 * N)
    if remainder == 0:
        print(f"Case #{t}: 0")
    else:
        x = (2 * N - remainder) % (2 * N)
        print(f"Case #{t}: {x}")
********************************************************************************
WRONG ANSWER!!

<expected>
'Case #1: 4
Case #2: 7
Case #3: 1
Case #4: -1
Case #5: 6
Case #6: -1
Case #7: 1000000002
'
</expected>
---
<got>
'Case #1: 1
Case #2: 3
Case #3: 1
Case #4: 2
Case #5: 6
Case #6: 0
Case #7: 0
'
</got>


Let's build a simple evaluation using weave to evaluate the zero-shot agent.
You'll quickly see how this simple evaluation framework can become very powerful and will scale to very complex workflows.
Our agent already takes care of running the code, evaluating the solution against the expected output for the sample test cases and returning the report in the model output.
We expect that the `test_report` is `"passed"` in the agent output so we can use that to evaluate the agent. 

But first we need to load all the problems and convert them to a more structured format. A good agent should be able to handle all the problems in the dataset.

In [12]:
# This is a simple depection of the evaluation.
# We expect the output to be `"passed"` for all the problems if the agent is working correctly.
examples = [{"problem": problem, "expected": "passed"} for problem in problems]


# A simple scorer that checks if the code generated by agent passed the test case
@weave.op
def scorer(expected: str, model_output: dict) -> dict:
    return {"passed": expected == model_output["test_report"]}


# This is a simple evaluation that checks if the code generated by agent passed the test
eval = weave.Evaluation(dataset=examples, scorers=[scorer])

Now we are ready to evaluate the zero-shot agent.
We will create a `weave.Model` instance for the zero-shot agent.
This will help us conduct robust experiments and comparisons by helping us track various settings and parameters for the agent.
For now, we will focus on the `LLM` and the `temperature` for the model.


In [13]:
# Nothing fancy here, just a model that takes in a problem and returns a solution


class ZeroshotAgent(weave.Model):
    model: str = FAST_LLM
    temperature: float = 0.0
    timeout: int = 30

    @weave.op
    async def predict(self, problem: Problem):
        return await zero_shot_solver(
            Problem(**problem),
            model=self.model,
            temperature=self.temperature,
            timeout=self.timeout,
        )

In [14]:
# Evaluate the zero shot agent for all the models and temperatures
LLM = FAST_LLM  # feel free to try STRONG_LLM
temperature = 0.5 # feel free to try other temperatures


zeroshot_agent = ZeroshotAgent(model=LLM, temperature=temperature, timeout=30)
zeroshot_results = await eval.evaluate(zeroshot_agent)

2024-09-05 17:02:07,554 : INFO : Drafting intial zero-shot solution
17:02:07 - LiteLLM:INFO: utils.py:2975 - 
LiteLLM completion() model= open-mistral-nemo-2407; provider = mistral
2024-09-05 17:02:07,561 : INFO : 
LiteLLM completion() model= open-mistral-nemo-2407; provider = mistral
2024-09-05 17:02:07,862 : INFO : Drafting intial zero-shot solution
17:02:07 - LiteLLM:INFO: utils.py:2975 - 
LiteLLM completion() model= open-mistral-nemo-2407; provider = mistral
2024-09-05 17:02:07,869 : INFO : 
LiteLLM completion() model= open-mistral-nemo-2407; provider = mistral
2024-09-05 17:02:08,178 : INFO : Drafting intial zero-shot solution
17:02:08 - LiteLLM:INFO: utils.py:2975 - 
LiteLLM completion() model= open-mistral-nemo-2407; provider = mistral
2024-09-05 17:02:08,183 : INFO : 
LiteLLM completion() model= open-mistral-nemo-2407; provider = mistral
2024-09-05 17:02:08,473 : INFO : Drafting intial zero-shot solution
17:02:08 - LiteLLM:INFO: utils.py:2975 - 
LiteLLM completion() model= open

Evaluated 1 of 5 examples

2024-09-05 17:02:20,852 : INFO : HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-05 17:02:21,190 : INFO : Draft solution result: "WRONG ANSWER!!\n\n<expected>\n'Case #1: 3\nCase #2: 1\nCase #3: 0\nCase #4: 199\nCase #5: 100\nCase #6: 1999999999999\n'\n</expected>\n---\n<got>\n'1\n0\n0\n50\n50\n500000000000\n'\n</got>"


Evaluated 2 of 5 examples

2024-09-05 17:02:21,535 : INFO : HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
17:02:21 - LiteLLM:INFO: utils.py:2975 - 
LiteLLM completion() model= open-mistral-nemo-2407; provider = mistral
2024-09-05 17:02:21,543 : INFO : 
LiteLLM completion() model= open-mistral-nemo-2407; provider = mistral
2024-09-05 17:02:22,338 : INFO : HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-05 17:02:22,686 : INFO : Draft solution result: 'failed: Traceback (most recent call last):\n  File "<string>", line 17, in <module>\nZeroDivisionError: integer division or modulo by zero\n'


...

2024-09-05 17:02:23,009 : INFO : HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-05 17:02:23,371 : INFO : Draft solution result: "WRONG ANSWER!!\n\n<expected>\n'Case #1: YES\nCase #2: NO\nCase #3: YES\nCase #4: YES\nCase #5: YES\nCase #6: YES\nCase #7: NO\n'\n</expected>\n---\n<got>\n'Case #1: YES\nCase #2: YES\nCase #3: YES\nCase #4: YES\nCase #5: YES\nCase #6: YES\nCase #7: NO\n'\n</got>"


Evaluated 4 of 5 examples

2024-09-05 17:02:34,006 : INFO : HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-05 17:02:34,379 : INFO : Draft solution result: 'failed: Traceback (most recent call last):\n  File "<string>", line 62, in <module>\n  File "<string>", line 21, in solve_problem\nTypeError: \'int\' object is not subscriptable\n'


Evaluated 5 of 5 examples

Evaluation summary
{'scorer': {'passed': {'true_count': 0, 'true_fraction': 0.0}}, 'model_latency': {'mean': 16.788401460647584}}

Once you have the results you should also be able to visit your weave dashboard to see the results.

## Reflection Agent



While the RAG agent is an improvement over the zero-shot agent, it's still not perfect.
It's still susceptible to hallucinations and incorrect solutions. 
One way to mitigate this is to use reflection.
We can use another LLM call to reflect on the solution and test results and improve it.
We can then use the improved solution to generate new few-shot examples and repeat the process in a loop until we converge to a solution or the iteration limit is reached.

Again, this is not the best approach to solve the problem and has a lot of room for improvement, but it should help us get towards a working solution.

Here are the reflection instructions we will provide to the LLM to reflect on the solution and test results, feel free to change the instructions to improve the agent's performance.

In [20]:
from agent import rag_solver
from retriever import Retriever

retriever = Retriever()

logger.info("Loading retriever ... this may take a while ...")
from agent import REFLECTION_INSTRUCTIONS, rework_solution

print(REFLECTION_INSTRUCTIONS)

2024-09-05 17:04:07,843 : DEBUG : Building index from IDs objects      
2024-09-05 17:04:08,785 : INFO : Loading retriever ... this may take a while ...


You are a world-class competitive programmer with a keen eye for detail and problem solving. 
Your expertise is in algorithms and data structures. 
You have incorrectly answered the following programming problem. 
Your task is to reflect on the problem, your solution, and the correct answer.
You will then use this information help you answer the same question in the future. 
First, explain why you answered the question incorrectly.
Second, list the keywords that describe the type of your errors from most general to most specific.
Third, solve the problem again, step-by-step, based on your knowledge of the correct answer.
Fourth, create a list of detailed instructions to help you correctly solve this problem in the future.
Finally, create a list of general advice to help you solve similar types of problems in the future.
Be concise in your response; however, capture all of the essential information.

{problem}
<incorrect_solution>
{incorrect_solution}
</incorrect_solution>
<test_report>

In [19]:
@weave.op
async def rag_solver_with_reflection(
        retriever: Retriever,
        problem: Problem,
        model: str = FAST_LLM,
        temperature: float = 0.0,
        max_iterations: int = 2,
        timeout: int = 10,
):
    num_iterations = 0
    test_report = "failed"
    solution = None
    while not test_report == "passed" and num_iterations < max_iterations:
        rag_result = await rag_solver(
            retriever=retriever,
            problem=problem,
            timeout=timeout,
            model=model,
            temperature=temperature,
        )
        solution = rag_result["solution"]
        test_report = rag_result["test_report"]
        if test_report == "passed":
            return rag_result
        rework_result = await rework_solution(
            problem=problem,
            incorrect_solution=solution,
            test_report=test_report,
            model=model,
            temperature=temperature,
            timeout=timeout,
        )
        solution = rework_result["solution"]
        test_report = rework_result["test_report"]
        if test_report == "passed":
            return {
                "solution": solution,
                "stage": "reflection",
                "test_report": test_report,
            }
        num_iterations += 1
    logger.info("Failed to generate a solution")
    return {"solution": solution, "stage": "failed", "test_report": test_report}

In [ ]:
reflection_result = await rag_solver_with_reflection(
    retriever, problem, max_iterations=2, timeout=30
)

print("*" * 80)
print(reflection_result["solution"].source_code)
print("*" * 80)
print(reflection_result["test_report"])

2024-09-05 10:31:03,780 : INFO : Drafting intial zero-shot solution
2024-09-05 10:31:16,477 : INFO : HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-05 10:31:30,492 : INFO : HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-05 10:31:40,511 : INFO : Draft solution result: "WRONG ANSWER!!\n\n<expected>\n'Case #1: 4\nCase #2: 7\nCase #3: 1\nCase #4: -1\nCase #5: 6\nCase #6: -1\nCase #7: 1000000002\n'\n</expected>\n---\n<got>\n'Case #1: -1\nCase #2: -1\nCase #3: -1\nCase #4: -1\nCase #5: -1\nCase #6: 1\nCase #7: -1\n'\n</got>"
2024-09-05 10:31:40,513 : INFO : Iterating on a RAG solution
2024-09-05 10:31:41,179 : INFO : Generating RAG solution:
2024-09-05 10:31:41,900 : INFO : Generating examplars:
2024-09-05 10:31:42,434 : INFO : HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-05 10:31:43,018 : INFO : HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-

********************************************************************************
def can_form_pairs(weights, target_weight, N):
    from collections import Counter
    count = Counter(weights)
    for weight in sorted(count.keys()):
        while count[weight] > 0:
            complement = target_weight - weight
            if complement in count and count[complement] > 0:
                if weight == complement:
                    if count[weight] < 2:
                        return False
                    count[weight] -= 2
                else:
                    count[weight] -= 1
                    count[complement] -= 1
            else:
                return False
    return True

def solve():
    import sys
    input = sys.stdin.read
    data = input().splitlines()
    T = int(data[0])
    results = []
    index = 1
    for case_number in range(1, T + 1):
        N = int(data[index])
        weights = list(map(int, data[index + 1].split()))
        index += 2
        tota

Great, now, we are ready to evaluate a more complex agent that uses reflection
This agent will try to solve the problem using the retriever
and if it fails, it will ask the model to reflect on the problem
and then re-work the solution
and repeat this process for a fixed number of iterations
or until the solution is correct or the iteration limit is reached

But the best part is that we can use the same evaluation framework we used for the zero-shot and RAG agent to evaluate the RAG reflection agent.

In [ ]:
class RAGReflectionAgent(weave.Model):
    retriever: Retriever
    max_iterations: int = 2
    timeout: int = 30
    model: str = STRONG_LLM
    temperature: float = 0.0

    @weave.op
    async def predict(self, problem: Problem):
        return await rag_solver_with_reflection(
            self.retriever,
            Problem(**problem),
            model=self.model,
            temperature=self.temperature,
            max_iterations=self.max_iterations,
            timeout=self.timeout,
        )

In [ ]:
# Evaluate the RAG reflection agent for all the models and temperatures
tasks = []
for LLM in eval_models:
    for temperature in eval_temperatures:
        rag_reflection_agent = RAGReflectionAgent(
            retriever=retriever, model=LLM, temperature=temperature, timeout=30
        )
        rag_reflection_results = eval.evaluate(rag_reflection_agent)
        tasks.append(rag_reflection_results)
rag_reflection_results = await asyncio.gather(*tasks)

2024-09-05 10:37:39,184 : INFO : Drafting intial zero-shot solution
2024-09-05 10:37:39,489 : INFO : Drafting intial zero-shot solution
2024-09-05 10:37:39,771 : INFO : Drafting intial zero-shot solution
2024-09-05 10:37:40,055 : INFO : Drafting intial zero-shot solution
2024-09-05 10:37:40,376 : INFO : Drafting intial zero-shot solution
2024-09-05 10:37:40,864 : INFO : Drafting intial zero-shot solution
2024-09-05 10:37:41,169 : INFO : Drafting intial zero-shot solution
2024-09-05 10:37:41,504 : INFO : Drafting intial zero-shot solution
2024-09-05 10:37:41,819 : INFO : Drafting intial zero-shot solution
2024-09-05 10:37:42,090 : INFO : Drafting intial zero-shot solution
2024-09-05 10:37:42,390 : INFO : Drafting intial zero-shot solution
2024-09-05 10:37:42,699 : INFO : Drafting intial zero-shot solution
2024-09-05 10:37:42,981 : INFO : Drafting intial zero-shot solution
2024-09-05 10:37:43,280 : INFO : Drafting intial zero-shot solution
2024-09-05 10:37:43,585 : INFO : Drafting intial

model_output failed

Traceback (most recent call last):
  File "/Users/tcapelle/miniforge3/envs/weave/lib/python3.11/site-packages/weave/flow/eval.py", line 168, in predict_and_score
    model_output = await async_call(model_predict, **model_predict_args)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/tcapelle/miniforge3/envs/weave/lib/python3.11/site-packages/weave/trace/op.py", line 333, in wrapper
    res, _ = await _execute_call(wrapper, call, *args, **kwargs)  # type: ignore
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/tcapelle/miniforge3/envs/weave/lib/python3.11/site-packages/weave/trace/op.py", line 213, in _call_async
    return handle_exception(e)
           ^^^^^^^^^^^^^^^^^^^
  File "/Users/tcapelle/miniforge3/envs/weave/lib/python3.11/site-packages/weave/trace/op.py", line 211, in _call_async
    res = await func(*args, **kwargs)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/sf/tgv7vcv96x557p38bvvp1ms40000

Predict and score failed

Traceback (most recent call last):
  File "/Users/tcapelle/miniforge3/envs/weave/lib/python3.11/site-packages/weave/flow/eval.py", line 287, in eval_example
    eval_row = await self.predict_and_score(model, example)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/tcapelle/miniforge3/envs/weave/lib/python3.11/site-packages/weave/trace/op.py", line 333, in wrapper
    res, _ = await _execute_call(wrapper, call, *args, **kwargs)  # type: ignore
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/tcapelle/miniforge3/envs/weave/lib/python3.11/site-packages/weave/trace/op.py", line 213, in _call_async
    return handle_exception(e)
           ^^^^^^^^^^^^^^^^^^^
  File "/Users/tcapelle/miniforge3/envs/weave/lib/python3.11/site-packages/weave/trace/op.py", line 211, in _call_async
    res = await func(*args, **kwargs)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/tcapelle/miniforge3/envs/weave/lib/python3.11/site-packages/weave/fl

...

2024-09-05 11:04:51,486 : INFO : HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-05 11:05:23,786 : INFO : Reworked solution result: 'timed out'
2024-09-05 11:05:23,792 : INFO : Failed to generate a solution
2024-09-05 11:05:23,800 : INFO : HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-05 11:05:26,023 : INFO : Reworked solution result: "WRONG ANSWER!!\n\n<expected>\n'Case #1: 5\nCase #2: -2\nCase #3: 0\nCase #4: -2\n'\n</expected>\n---\n<got>\n'Case #1: 0\nCase #2: -2\nCase #3: 0\nCase #4: -2\n'\n</got>"
2024-09-05 11:05:26,025 : INFO : Failed to generate a solution


Evaluated 5 of 5 examples

Evaluation summary
{'scorer': {'passed': {'true_count': 2, 'true_fraction': 0.4}}, 'model_latency': {'mean': 1029.150792837143}}

Evaluated 5 of 5 examples

Evaluation summary
{'scorer': {'passed': {'true_count': 2, 'true_fraction': 0.4}}, 'model_latency': {'mean': 1120.1171435832978}}

Okay, that completes the demo!

Key takeaways from this demo:
1. We tried to solve some challenging competitive programming problems using LLM agents.
2. We tried three different agents:
    - Zero-shot agent
    - RAG agent
    - RAG reflection agent
3. We used Weave to evaluate the agents and compare their performance.

We hope you found this demo useful and interesting and that it gave you some ideas on how to use LLM agents to solve challenging problems.
